In [ ]:
from datascience import *
import numpy as np
%matplotlib inline

In [ ]:
scores = Table.read_table("scores.csv")
scores.show(5)

In [ ]:
scores.scatter('Midterm 1', 'Midterm 2', colors='Mentored')

In [ ]:
scores.hist('Midterm 1', group='Mentored', bins=np.arange(0, 41, 5), normed=False)

In [ ]:
examples = scores.where("Mentored", False)

def predict_mt2(mt1):
    near = examples.where("Midterm 1", are.between_or_equal_to(mt1-2, mt1+2))
    return near.column("Midterm 2").mean()

mt1_scores = examples.select("Midterm 1").sort(0, distinct=True)
predictions = mt1_scores.with_column("Predicted MT 2", 
                                     mt1_scores.apply(predict_mt2, "Midterm 1"))
t = scores.join("Midterm 1", predictions)
t.drop("Mentored").scatter(0)

In [ ]:
u = t.with_column("Improvement", t.column("Midterm 2") - t.column("Predicted MT 2"))
u.hist("Improvement", bins=np.arange(-30, 31, 5), group="Mentored", unit="point")

In [ ]:
def of_at_least_5(values):
    return sum(values >= 5) / len(values)

u.select(2, 4).group("Mentored", of_at_least_5).set_format(1, PercentFormatter)

In [ ]:
u.group("Mentored", np.mean)

In [ ]:
def mean_ci(observations):
    means = []
    for i in np.arange(2000):
        means.append(observations.sample().column("Improvement").mean())
    lower, upper = percentile(2.5, means), percentile(97.5, means)
    print("Mean improvement:", observations.column("Improvement").mean())
    print("95% CI of mean improvement:", lower, "to", upper)

mentored = u.where("Mentored", True)
mean_ci(mentored)

In [ ]:
mean_ci(mentored.where("Midterm 1", are.below(20)))

In [ ]:
mean_ci(mentored.where("Midterm 1", are.between(20, 30)))

In [ ]:
mean_ci(mentored.where("Midterm 1", are.above_or_equal_to(30)))